In [1]:

import numpy as np
import pandas as pd
import datetime
from scipy.stats import uniform, randint
import math
from sklearn.datasets import load_breast_cancer, load_diabetes, load_wine
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split,StratifiedKFold

from sklearn.datasets import fetch_openml
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score,classification_report
from sklearn.model_selection import train_test_split
import time
from sklearn.neural_network import MLPClassifier

/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [11]:
df = 'newd.csv'

Data = pd.read_csv(df)[['date','exdate','delta','impl_volatility']] #try using more features later
Data['date'] = pd.to_datetime(Data['date'])
Data['exdate'] = pd.to_datetime(Data['exdate'])


sdate = datetime.datetime(2018,2,28)
edate = datetime.datetime(2023,2,28)


Data = Data[Data['date'] < edate] #change this line if we need more complicated set of data
Data = Data[Data['date'] >= sdate]

Data['time_to_maturity'] = (Data['exdate']-Data['date']).astype('int64')/10**9/3600/24
Data = Data.dropna()

Data
Data = Data.dropna()

Data = Data.sort_values(by='date')
Data['impl_volatility'] = np.log(Data['impl_volatility'])
print(Data)



               date     exdate     delta  impl_volatility  time_to_maturity
0        2018-02-28 2018-03-16  0.995644         0.687412              16.0
8079     2018-02-28 2018-04-04  0.751733        -1.591962              35.0
8080     2018-02-28 2018-04-04  0.744376        -1.604212              35.0
8081     2018-02-28 2018-04-04  0.736836        -1.617117              35.0
8082     2018-02-28 2018-04-04  0.728928        -1.629789              35.0
...             ...        ...       ...              ...               ...
21457096 2023-02-27 2023-02-28 -0.000468        -0.037699               1.0
21457097 2023-02-27 2023-02-28 -0.000472        -0.046069               1.0
21457098 2023-02-27 2023-02-28 -0.000480        -0.062996               1.0
21456993 2023-02-27 2023-02-28  0.000654        -0.372365               1.0
21468994 2023-02-27 2023-12-29 -0.001588        -0.386937             305.0

[19198643 rows x 5 columns]


In [12]:
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Futures(X) we would be using and response variable(y)

features = ['delta', 'time_to_maturity']
X = Data[features]
y = Data['impl_volatility']

# Add a constant term for the linear regression
X = sm.add_constant(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the linear regression model
model = sm.OLS(y_train, X_train)
result = model.fit()

# Print the summary of the linear regression model
print(result.summary())

# Make predictions on the test set
y_pred = result.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse:.4f}')
print(f'R-squared: {r2:.4f}')


                            OLS Regression Results                            
Dep. Variable:        impl_volatility   R-squared:                       0.070
Model:                            OLS   Adj. R-squared:                  0.070
Method:                 Least Squares   F-statistic:                 5.797e+05
Date:                Tue, 28 Nov 2023   Prob (F-statistic):               0.00
Time:                        10:36:14   Log-Likelihood:            -1.2384e+07
No. Observations:            15358914   AIC:                         2.477e+07
Df Residuals:                15358911   BIC:                         2.477e+07
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -1.3322      0.000  

In [14]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler


# Futures(X) we would be using and response variable(y)
features = ['delta', 'time_to_maturity']
X = Data[features]
y = Data['impl_volatility']

# Standardize features (optional but can be beneficial for some algorithms)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Create and train the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse:.4f}')
print(f'R-squared: {r2:.4f}')

# Print the coefficients and intercept
print(f'Coefficients: {model.coef_}')
print(f'Intercept: {model.intercept_}')


Mean Squared Error: 0.2943
R-squared: 0.0696
Coefficients: [ 0.14186466 -0.04650144]
Intercept: -1.3293617199557357


In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Select relevant features and target variable
features = ['delta', 'time_to_maturity']
X = Data[features]
y = Data['impl_volatility']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the Random Forest model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse:.4f}')
print(f'R-squared: {r2:.4f}')


Mean Squared Error: 0.1511
R-squared: 0.5222



## Code below is not working for some reason. Ignore beyond this line!

In [7]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
df = 'newd.csv'

Data = pd.read_csv(df)[['date','exdate','delta','impl_volatility']] #try using more features later
Data['date'] = pd.to_datetime(Data['date'])
Data['exdate'] = pd.to_datetime(Data['exdate'])


sdate = datetime.datetime(2018,2,28)
edate = datetime.datetime(2023,2,28)


Data = Data[Data['date'] < edate] #change this line if we need more complicated set of data
Data = Data[Data['date'] >= sdate]

Data['time_to_maturity'] = (Data['exdate']-Data['date']).astype('int64')/10**9/3600/24
Data = Data.dropna()

Data

X = Data[['date', 'exdate', 'delta']]
y = Data['impl_volatility']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the input features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build a simple neural network
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_dim=X_train.shape[1]),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Train the model
model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set
y_pred_nn = model.predict(X_test_scaled)
accuracy_nn = accuracy_score(y_test, y_pred_nn.round())
print("Neural Network Model Accuracy:", accuracy_nn)


SyntaxError: invalid syntax (pywrap_tensorflow_internal.py, line 114)